# Generative Spaces (ABM)

In this workshop we will lwarn how to construct a ABM (Agent Based Model) with spatial behaviours, that is capable of configuring the space.

## 0. Initialization

### 0.1. Load required libraries

In [7]:
import os
import topogenesis as tg
import pyvista as pv
import trimesh as tm
import numpy as np
np.random.seed(0)

### 0.2. Define the Neighborhood (Stencil)

In [9]:
# creating neighborhood definition
stencil = tg.create_stencil("von_neumann", 1, 1)
print(stencil)

[[[0 0 0]
  [0 1 0]
  [0 0 0]]

 [[0 1 0]
  [1 1 1]
  [0 1 0]]

 [[0 0 0]
  [0 1 0]
  [0 0 0]]]


## 1. Agents

### 1.1. Define the Agents Class

In [2]:
# agent class
class agent():
    def __init__(self, origin, stencil, id):

        # define the origin attribute of the agent and making sure that it is an intiger
        self.origin = np.array(origin).astype(int)
        # define old origin attribute and assigning the origin to it as the initial state
        self.old_origin = self.origin
        # define stencil of the agent
        self.stencil = stencil
        #define agent id
        self.id = id

    # definition of walking method for agents
    def walk(self, env):
        # find available spaces
        #######################

        # retrieve the list of neighbours of the agent based on the stencil
        neighs = env.availibility.find_neighbours_masked(self.stencil, loc = self.origin)
        # find availability of neighbours
        neighs_availibility = env.availibility.flatten()[neighs]
        # separate available neighbours
        free_neighs = neighs[neighs_availibility==1]
        # retrieve the value of each neighbour
        free_neighs_value = env.value.flatten()[free_neighs]
        # find the neighbour with maximum my value
        selected_neigh = free_neighs[np.argmax(free_neighs_value)]
        
        # update information
        ####################

        # set the current origin as the ol origin
        self.old_origin = self.origin
        # update the current origin with the new selected neighbour
        self.origin = np.array(np.unravel_index(selected_neigh, env.availibility.shape)).flatten()

### 1.2. Initialize and scatter agents randomly

In [3]:
# randomly scattering the agents
# TODO: only choose from available voxels
selected_cells = np.random.choice(avail_lattice.size, 5)
agent_ind = np.array(np.unravel_index(selected_cells, avail_lattice.shape))

agents= []
# creating agent objects
for id, ind in enumerate(agent_ind.T.tolist()):
    myagent = agent(ind, stencil, id+1)
    agents.append(myagent)

## 2. Setup the Environment

### 2.1. Load the envelope lattice as the avialbility lattice

In [ ]:
# loading the lattice from csv
lattice_path = os.path.relpath('../data/voxelized_envelope.csv')
avail_lattice = tg.lattice_from_csv(lattice_path)

### 2.2. Load the Sun Access lattice as the value field

In [11]:
# loading the lattice from csv
lattice_path = os.path.relpath('../data/sun_access.csv')
sun_acc_lattice = tg.lattice_from_csv(lattice_path)

### 2.3. Define Environment Class

In [4]:
# environment class
class environment():
    def __init__(self, lattices, agents):
        self.availibility = lattices["availibility"]
        self.value = lattices["sun_access"]
        self.agent_origin = self.availibility * 0
        self.agents = agents
        self.update_agents()
    
    def update_agents(self):
        for a in self.agents:
            # making previous position available
            self.availibility[tuple(a.old_origin)] = self.availibility[tuple(a.old_origin)] * 0 + 1
            # removing agent from previous position
            self.agent_origin[tuple(a.old_origin)] *= 0
            # making the current position unavailable
            self.availibility[tuple(a.origin)] *= 0
            # adding agent to the new position 
            self.agent_origin[tuple(a.origin)] = a.id
    
    def walk_agents(self):
        # iterate over egents and perform the walk
        for a in self.agents:
            a.walk(self)
        # update the agent states in environment
        self.update_agents()

### 2.4. Create the environment

In [6]:
# name the lattices
env_lattices = {"availibility": avail_lattice,
                "sun_access": sun_acc_lattice}
# initiate the environment
env = environment(env_lattices, agents)



[array([0, 4, 8]), array([0, 5, 2]), array([0, 7, 1]), array([0, 7, 4]), array([0, 7, 4])]
[array([0, 4, 7]), array([0, 5, 3]), array([0, 6, 1]), array([0, 6, 4]), array([0, 6, 4])]
[array([0, 4, 6]), array([0, 4, 3]), array([0, 6, 2]), array([0, 5, 4]), array([0, 5, 4])]
[array([0, 4, 5]), array([0, 4, 4]), array([0, 5, 2]), array([0, 4, 4]), array([0, 4, 4])]
[array([0, 3, 5]), array([0, 3, 4]), array([0, 5, 3]), array([0, 3, 4]), array([0, 3, 4])]
[array([0, 4, 5]), array([0, 4, 4]), array([0, 4, 3]), array([0, 4, 4]), array([0, 4, 4])]
[array([0, 3, 5]), array([0, 3, 4]), array([0, 3, 3]), array([0, 3, 4]), array([0, 3, 4])]
[array([0, 4, 5]), array([0, 4, 4]), array([0, 4, 3]), array([0, 4, 4]), array([0, 4, 4])]
[array([0, 3, 5]), array([0, 3, 4]), array([0, 3, 3]), array([0, 3, 4]), array([0, 3, 4])]
[array([0, 4, 5]), array([0, 4, 4]), array([0, 4, 3]), array([0, 4, 4]), array([0, 4, 4])]


## 3. Run the Simulation

In [ ]:
for i in range(10):
    # print(env.availibility)
    # print(env.agent_origin)
    agn_org = [a.origin for a in env.agents]
    print(agn_org)
    env.walk_agents()

### Credits

In [ ]:
__author__ = "Shervin Azadi "
__license__ = "MIT"
__version__ = "1.0"
__url__ = "https://github.com/shervinazadi/spatial_computing_workshops"
__summary__ = "Spatial Computing Design Studio Workshop on Agent Based Models for Generative Spaces"